In [168]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [169]:
#read
unsw = pd.read_csv("../datasets/UNSW_NB15_training-set.csv")

In [170]:
# Load data
"""Load unsw dataset"""
y = unsw.iloc[:,-1:]
#one-hot-encode parameters
proto = pd.get_dummies(unsw['proto'])
state = pd.get_dummies(unsw['state'])
service = pd.get_dummies(unsw['service'])
service = service.iloc[:,1:]
raw_data = unsw.values

#remove encoded parameters and add one hot
for x in ['proto', 'state', 'service','label','attack_cat']:
    unsw = unsw.drop(x,axis = 1)
    
for x in [proto, state, service]:
    unsw = unsw.join(x)
    
X = unsw.iloc[:,:-2]

### First Node

In [171]:
X1 = X[:30000] 
y1 = y[:30000]

In [16]:
parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
              {'penalty':['none', 'elasticnet', 'l1', 'l2']},
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]
LogReg = LogisticRegression()
clf = GridSearchCV(LogReg, parameters)
clf.fit(X1, y1)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tk/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/.local/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was pa

GridSearchCV(estimator=LogisticRegression(),
             param_grid=[{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']},
                         {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100]}])

In [20]:
clf.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'auto',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': LogisticRegression(),
 'n_jobs': None,
 'param_grid': [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
  {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
  {'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

In [172]:
clf1 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(X1,y1)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [173]:
print(clf1.coef_)
print(clf1.intercept_)

[[-6.70321748e-07  2.27139200e-10  1.73047152e-09  3.60951522e-09
   1.89288840e-06  5.53080222e-06  3.12348534e-06  1.91292298e-08
   1.58786179e-08  3.54193097e-08 -4.46757971e-06  5.59139743e-10
   1.94260585e-09 -3.49978945e-08 -6.55382751e-09  2.50057422e-07
  -4.36584094e-08  2.39195571e-09  2.68518368e-10  3.01307905e-10
   3.07833939e-09  7.19834465e-12  3.33398509e-12  3.86435956e-12
   3.47244405e-09  1.11799051e-08  2.04182575e-11  9.90769263e-07
   6.62572825e-10  1.51302685e-10  5.25820413e-10  6.69434720e-10
   5.46927437e-10  9.48064223e-10  5.48735706e-13  4.82392258e-13
   2.12390128e-11  5.59553177e-10  6.63844903e-10 -4.30782485e-13
   1.91226820e-14 -1.35903916e-18  1.03069437e-15  3.23385299e-14
   1.92946470e-14  1.91228061e-14 -8.83058796e-13  1.91227016e-14
   3.93607220e-17  1.03289192e-17  1.21852829e-14  4.09299184e-13
   1.91224311e-14  1.09029831e-17  1.21855698e-14 -1.35903920e-18
   1.21851952e-14  1.03293267e-17  1.03293803e-17  2.01849014e-14
   1.91228

### Second learner

In [174]:
X2 = X[30000:60000]
y2 = y[30000:60000]

In [175]:
clf2 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=131, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(X2,y2)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [176]:
print(clf2.coef_)
print(clf2.intercept_)

[[ 8.38181121e-06  1.73692897e-10  2.16965259e-09  3.30964598e-09
   3.06153635e-06  5.63122588e-06  1.60711823e-05  4.50085388e-08
   1.93768348e-08 -2.50223184e-09 -4.89279160e-06  9.97858452e-10
   1.85908539e-09 -3.99821322e-07 -3.61655382e-10  1.74307579e-06
   1.37463130e-09  2.41884814e-09  5.31710213e-12  5.11635146e-12
   2.59930648e-09  3.18871520e-12  5.90924376e-13  2.59779082e-12
   3.49412355e-09  1.42836150e-08 -5.25441812e-12  3.38955473e-07
   9.21119898e-10  3.04698266e-10  1.53610139e-10  4.30183908e-10
   4.75657286e-10 -5.15858244e-11 -3.67807881e-12 -3.75127818e-12
  -1.67706870e-11  3.53795797e-10  8.07231774e-10 -7.00927459e-12
   2.72632065e-13  1.43186882e-13  2.65803790e-13  4.49397998e-13
   2.73868052e-13  2.35892149e-13 -8.06049981e-12  2.20924384e-13
   2.73863631e-13  1.76690277e-13  3.79734272e-14  3.50970450e-13
   2.65818273e-13  2.65780911e-13  2.65769968e-13  0.00000000e+00
   0.00000000e+00  1.43175668e-13  9.82637674e-14  1.76728744e-13
   1.76698

### Test local models

In [177]:
X_test = X[60000:]
y_test = y[60000:]

In [178]:
y_hat1 =clf1.predict(X_test)

In [179]:
y_hat2 =clf2.predict(X_test) 

In [180]:
accuracy_score(y_test,y_hat1)

0.2987641053197206

In [181]:
accuracy_score(y_test,y_hat2)

0.254253985312556

In [166]:
y_hat1

array([1, 1, 1, ..., 0, 0, 1])

In [167]:
y_hat2

array([0, 0, 0, ..., 1, 1, 1])

In [115]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable


In [196]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(188, 100)
        self.layer2 = nn.Linear(100, 30)
        self.layer3 = nn.Linear(30, 1)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        return x


In [130]:
model     = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()

In [197]:
import tqdm
def train(X_train, y_train):
    EPOCHS  = 100

    loss_list     = np.zeros((EPOCHS,))
    accuracy_list = np.zeros((EPOCHS,))

    for epoch in tqdm.trange(EPOCHS):
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        loss_list[epoch] = loss.item()
        
        # Zero gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [198]:
X1_torch = torch.tensor(X1.values.astype(np.float32))
y1_torch = torch.tensor(y1.values.astype(np.float32))


In [199]:
X2_torch = torch.tensor(X2.values.astype(np.float32))
y2_torch = torch.tensor(y2.values.astype(np.float32))

In [200]:

train(X1_torch,y1_torch)

100%|██████████| 100/100 [00:07<00:00, 13.41it/s]


In [202]:
X_test

tensor([[6.0001e+04, 5.7045e-01, 1.0000e+01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [6.0002e+04, 1.1000e-05, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [6.0003e+04, 7.0000e-06, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [8.2330e+04, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.2331e+04, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.2332e+04, 9.0000e-06, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [203]:
y_pred_list = []
predictions_train = []
predictions_test =  []
with torch.no_grad():
    predictions_test = model(X_test)

In [204]:
test = (predictions_test.numpy() > 0.5).astype(int)

In [205]:
train(X2_torch,y2_torch)

100%|██████████| 100/100 [00:07<00:00, 12.85it/s]


In [206]:
predictions_train = []
predictions_test =  []
with torch.no_grad():
    predictions_test = model(X_test)

In [207]:
accuracy_score(y_test,test)

0.2609260254343543